In [35]:
import string
import pandas as pd
import fitz
import os

**"def add_page"** is for adding pages to all words

In [36]:
def add_page(in_put, value):
    lst = []
    out = []
    index = 0

    for i in range(0, len(in_put)):
        if(i < len(in_put)-1):
            out += [[]]
            lst += in_put[i]
            lst.insert( 8, value+1)
            out[index] = lst
            index += 1
            lst = []
        else:
            out +=[[]]
            lst += in_put[i]
            lst.insert( 8, value+1)
            out[index] = lst

    return out

**"def get_all_text"** is getting all text from given pdf file.
* WC stands for word with coordinate.
* out is the return value of each words characteristic by PyMuPDF feature.
  The order of the output shows like [x0, y0, x1, y1, word, block_no, line_no, word_no, page].


In [37]:
def get_all_text(data):
    page_count = data.page_count
    out = []
    for i in range(0, page_count):
        load_page = data.load_page(i)
        #WC stand for word with coordinate
        WC = load_page.get_text('words')
        agw = add_page(WC, i)
        out += agw
    
    return out

**"def word_line"** is merging the words by block_no and line_no

In [38]:
def word_line(in_put):
    #in_put += [in_put[-1]]
    index = 0
    lst = []
    out = []
    for i in range(0, len(in_put)-1):
        # in_put[i][7] == word_no
        if(in_put[i][7]+1 == in_put[i+1][7]):
            lst += [in_put[i][4]]  
        else:
            if(in_put[i+1][7] == 0):
                lst += [in_put[i][4]]
                lst = [' '.join(lst)]
                lst += [in_put[i][5], in_put[i][6], in_put[i][7],in_put[i][8]]
                lst.insert( 0, in_put[i-in_put[i][7]][0])
                lst.insert( 1, in_put[i-in_put[i][7]][1])
                lst.insert( 2, in_put[i][2])
                lst.insert( 3, in_put[i][3])
            out += [[]]
            out[index] += lst
            index += 1
            lst = []
    #print(out)
    return out

**"def Same_y"** uses for rearrange the order of words after extraction.
PyMuPDF extracts text by using the coordinate of each word.
Sometimes the y coordinates in the same line will be a little different.
If one of the words in the same line has the smaller y position, It means that word will be extracted before the higher Y position words.

example: A & B in the same line 
y0 of A ==1, y0 of B == 0.9 
the order will be changed to B A since the y is more sensitive condition of the coming reorder work.
This function is to make the texts in the same line have the same y.

In [39]:
def Same_y(df):
    fill = [0,9999,0,9999,0,0,0,0,0]
    df += [fill]
    num = 0
    big_y = 99999
    highery = big_y
    
    for i in range(0, len(df)-1):        
        # Smaller Y  == higher position in the file 
        # df[i][1] == Y0, df[i][3] == Y1
        # if the average Y of text i in the between of the Ys of text i-1, we say they are the same line 
        if((df[i][1]+df[i][3])/2 > df[i+1][1]) & ((df[i][1]+df[i][3])/2 < df[i+1][3]) & (df[i][1] < highery):
            highery = df[i][1]
            num += 1
            #print(num)
        elif((df[i][1]+df[i][3])/2 > df[i+1][1]) & ((df[i][1]+df[i][3])/2 < df[i+1][3]):
            num += 1
            #print(num)
        # if the average Y of the text i is smaller than the text i+1, we know the i+1 is next line 
        #elif((df[i][1]+df[i][3])/2 < df[i+1][1]) or (i == len(df)-1):
        else:
            if(num == 0):
                pass
            else:
                for j in range(0, num+1):
                    df[i-j][1] = highery
                num = 0
                highery = big_y
    df.pop(-1)
    

    return df 

**"def get_fuel_x1"** is getting the X1 coordinate by using **"Fuel"** keyterm to be the reference of left and right column.
There are various words of "fuel" across all TCDSs. These are all keywords of "Fuel" keyterm that our team found based on TCDSs we have
*Fuel
*fuel:
**fuel
*fuel.
*fuel*

If users have new TCDSs with other variations of "Fuel" more than list above,
you can add more to get the coding to recognize the new variation by adding
or (df[i][4].lower() == '---your new variation of "Fuel"---')

In [40]:
def get_fuel_x1(df):
    for i in range(0,len(df)):
        if(df[i][4].lower() == 'fuel') or (df[i][4].lower() == 'fuel:') or (df[i][4].lower() == '*fuel') or (df[i][4].lower() == 'fuel.'):
            #print(df[i][2])
            return df[i][2]
        else:
            if(df[i][4].lower() == 'engine') or (df[i][4].lower() == 'engines') or (df[i][4].lower() == 'engine:') or (df[i][4].lower() == 'engines:') or (df[i][4].lower() == 'engine.') or (df[i][4].lower() == 'engines.') or (df[i][4].lower() == 'engines and'):
                return df[i][2]

**"def detect_khi"** is defining the word lines' categories of (key_term, header, info)

In [41]:
def detect_khi(df):
    index = 0
    lst = []
    out = []
    for i in range(0,len(df)):
        ## old one add & (in_put[i][6] == 0) in both if & elif
        if(df[i][0] <= fuel_x1) & (df[i][7] < 5 ):
            lst += df[i]
            lst.insert(8, 'key_term')
            out += [[]]
            out[index] = lst
            lst = []
            index += 1
        elif(df[i][0] <= fuel_x1) & (df[i][4].lower().find('model') >= 0):
            lst += df[i]
            lst.insert(8, 'header')
            out += [[]]
            out[index] = lst
            lst = []
            index += 1
        else:
            lst += df[i]
            lst.insert(8, 'info')
            out += [[]]
            out[index] = lst
            lst = []
            index += 1

    #print(out_put)
    return out

**def Drop_blw** is dropping some columns (block_no, line_no, word_no) that we don't use it anymore after definding categories.

In [42]:
def Drop_blw(df):
    # df == [x0, y0, x1, y1, word, block_no, line_no, word_no, category, page]    
    lst = []
    index = 0
    out = []
    for i in range(0, len(df)):
        lst += [df[i][0],df[i][1],df[i][2],df[i][3],df[i][4],df[i][8],df[i][9]]
        out += [[]]
        out[index] = lst
        index += 1
        lst = []
    # out == [x0, y0, x1, y1, word, category, page] 
    return out

**def hder** is used for ensure all model header will be collected in dataframe as the header.
Sometimes the header's number will be detected as key term, this function is used to solve it.
example: real header == I. Model xyz....  and the df might be ['I.', 'key_term'], ['Model xyz...', 'header']
hder function is to catch 'I.' form keyterm to header.  

In [43]:
def hder(df):
    for i in range(0, len(df)-1):
        if(df[i+1][5] == "header") & (int(df[i][1]) == int(df[i+1][1])):
            df[i][5] = "header"
    return df

**def firsty1** is finding first line of following pages.
It is important to let the coding know where to starting to extract.
Most of TCDSs, when they start the next page, on the top left-middle-right will have the page no, TCDS no and revision.
We consider them as the noises.

In [44]:
def firsty1(df):
    lst = []
    page = 1

    for i in range(0, len(df)-1):
        if(df[i][6] == page+1):
            lst += [df[i][3]]
            page += 1
    #print(lst)
    return lst

**def drop_fline** is dropping first line of following pages.

In [45]:
def drop_fline(df, fy):
    lst = []
    if len(fy) == 0:
        pass
    else:
        y1max = max(fy)
        for i in range(0, len(df)):
            #df[i][6] == page_no
            if(df[i][6] > 1) & (df[i][1] < y1max):
                lst += [i]
                #print(df[i][4])
#    print(lst)
    lst.reverse()
#    print(lst)
    for j in range(0, len(lst)):
        df.pop(lst[j])
    
    return df

**def clean_cont** is dropping the lines which contain of Cond't stuff for model header.
We consider the cond't stuff as the noise that will be better to remove.
There are various words of "cond't" across all TCDSs. These are all keywords of "cond't" keyterm that our team found based on TCDSs we have
*contd
*(cont’d)
*(continued)
*cont d
*cont

In [46]:
def clean_cont(df):

    lst = [] 

    for i in range(0, len(df)):
        if(df[i][0] <= fuel_x1):
            if(df[i][4].lower().find('contd') > -1) or (df[i][4].lower().find('(cont’d)') > -1) or (df[i][4].lower().find('(continued)') > -1) or (df[i][4].lower().find('cont d') > -1) or (df[i][4].lower().find('cont') > 0):
                lst += [i]

    #print(lst)
    lst.reverse()
    for j in range(0, len(lst)):
        df.pop(lst[j])
    
    return df

**def all_header** is used to make sure the left side has all header or keyterm.
Some TCDSs have model headers with multiple lines so this function lets all the lines after model header to the first keyterm(which always be Engine") to be defined as model header too.

In [47]:
def all_header(df, am):
    if(am == -1):
        am = len(df)
    else:
        pass
    
    for i in range(1 , am):
        if(df[i-1][5] == 'header'):
            if(df[i][4].lower() == 'engine') or (df[i][4].lower() == 'engines'):
                pass
            else:
                df[i][5] = 'header'
    return df

**def allmodel_is_header** is defining all model section as the model header.

In [48]:
def allmodel_is_header(df):
    
    for i in range(0, len(df)):
        if(df[i][5] == 'key_term') & (df[i][4].lower().find('all') > 0):
            if(df[i][4].lower().find('model') > 0) or (df[i][4].lower().find('models') > 0 ):
                df[i][5] = 'header'

    
    return df

**def df_rev** is extracting the top-right block at the first page of TCDSs and collect it in the 1st row in csv file.

In [49]:
def df_rev(df):
    fkt = -1
    for i in range(0, len(df)):
        if(df[i][5] == 'key_term') & (fkt < 0):
            fkt = i
#    print(fkt)        
    for i in range(0, fkt):
        if(df[i][0] > 300):
            df[i][5] = 'rev'
    
    return df

**def merge_all** is merging everything according all above conditions together in each categories.

In [50]:
def merge_all(df):
    df += [[0,0,0,0,0,0,0]]
    lst = []
    out = []
    index = 0
    num = 0
    for i in range(0, len(df)-1):
        if(df[i][5] == 'header'):
            if(df[i+1][5] == 'header'):
                lst += [df[i][4]]
                num += 1
            else:
                out += [[]]
                lst += [df[i][4]]
                lst = [' \n '.join(lst)]
                lst += [df[i][5]]
                lst.insert( 0, df[i-num][6])
                lst.insert( 1, df[i][1])
                lst.insert( 2, df[i][6])
                lst.insert( 3, df[i][3])
                # out put will be [start page, y start, end page, y end, text, class]
                out[index] = lst
                num = 0
                index += 1
                lst = []

        elif(df[i][5] == 'key_term'):
            out += [[]]
            lst += [df[i][6], df[i][1], df[i][6], df[i][3], df[i][4], df[i][5]]
            out[index] = lst
            index += 1
            lst = []
        
        elif(df[i][5] == 'info'):
            if(df[i+1][5] == 'info'):
                lst += [df[i][4]]
                num += 1
            else:
                out += [[]]
                lst += [df[i][4]]
                lst = [' \n '.join(lst)]
                lst += [df[i][5]]
                lst.insert( 0, df[i-num][6])
                lst.insert( 1, df[i][1])
                lst.insert( 2, df[i][6])
                lst.insert( 3, df[i][3])
                # out put will be [start page, y start, end page, y end, text, class]
                out[index] = lst
                num = 0
                index += 1
                lst = []
                
        elif(df[i][5] == 'rev'):
            if(df[i+1][5] == 'rev'):
                lst += [df[i][4]]
                num += 1
            else:
                out += [[]]
                lst += [df[i][4]]
                lst = [' \n '.join(lst)]
                lst += [df[i][5]]
                lst.insert( 0, df[i-num][6])
                lst.insert( 1, df[i][1])
                lst.insert( 2, df[i][6])
                lst.insert( 3, df[i][3])
                # out put will be [start page, y start, end page, y end, text, class]
                out[index] = lst
                num = 0
                index += 1
                lst = []
    return out

**def prep_final_output** is adding TCDS revision and model header in the first column.
Then if the keyterm is found, store it in the first column, follow by its info.
In case, there is keyterm without info, that keyterm will be dropped.

In [51]:
def prep_final_output(df):
    df += [[0,0,0,0,0,0]]
    index = 0
    lst = []
    out = []
    
    for i in range(0, len(df)-1):
        if(df[i][5] == 'rev') or (df[i][5] == 'header'):
            out += [[]]
            lst += [df[i][4], '']
            out[index] = lst
            
            lst = []
            index += 1
        
        elif(df[i][5] == 'key_term') & (df[i+1][5] == 'info'):
            lst += [df[i][4]]
        elif(df[i][5] == 'info') & (len(lst) > 0):
            out += [[]]
            lst += [df[i][4]]
            out[index] = lst
            
            lst =[]
            index += 1

    return out

**Loop function**

In [53]:

path = "/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds"
list_files = os.walk(path)
dfinal = pd.DataFrame() 
a = []
nof = []
scan = []
for root, dirs, files in os.walk(path):
    for file in files:
        if not file.startswith("."):
            p = os.path.join(root,file)
            print(p)
            pdf = fitz.open(p)
            if pdf == None:
                scan.append(file)
                pass
            else:
                all_text = get_all_text(pdf)
                merged_words = word_line(all_text)
#-----------------------------------
# Sometimes the y coordinates in the same line will be a little different,
# the smaller y, the higher position
# example: A & B in the same line 
# y0 of A ==1, y0 of B == 0.9 
# the order will be changed to B A since the y is more sensitive condition of the coming reorder work
# This function is to make the texts in the same line have the same y
                Sy = Same_y(merged_words)

#-------------------------------------
#sorted key + lambda 
# ref: https://blog.csdn.net/w417950004/article/details/86253721
# reorder the word lines by coordinates (page > y0 > x0)
                St = sorted(Sy, key = lambda x:(int(x[8]), float(x[1]), float(x[0])))

                fuel_x1 = get_fuel_x1(St)
                if fuel_x1 == None:
                    nof.append(file)
                    pdf.close()
                else:
                    khi = detect_khi(St)
#for i in range(0, len(khi)):
#    if(khi[i][8] == 'header'):
#        print(khi[i][4])
#Cleaned khi
                    Ckhi = Drop_blw(khi)

#-----------------------------------------------------------------------------
# Sometimes the header's number will be detect as key term, this function is used to solve it.
#example: real header == I. Model xyz....  and the df might be ['I.', 'key_term'], ['Model xyz...', 'header']
# hder function is to catch 'I.' form keyterm to header.       
                    Ckhi = hder(Ckhi)


#-----------------------
#clean the page stuff
#fy1 function will retun the y1 of the first lines
                    fy1 = firsty1(Ckhi)
#use the max value to clean the tc titles which has been detect as KT
                    drop_fline(Ckhi, fy1)
#-----------------------------------------------------------------------------
                    C_cont = clean_cont(Ckhi)
#---------------------------------------------------------------------------------------------------
#drop punctuations of KT
#https://datagy.io/python-remove-punctuation-from-string/
                    for i in range(0, len(C_cont)-1):
                        if(C_cont[i][5] == 'key_term'):
                            C_cont[i][4] = C_cont[i][4].translate(str.maketrans('', '', string.punctuation))
#-------------------------------------------------------------------------------------------------
                    amh = allmodel_is_header(C_cont)
#-------------------------------------------------------------------------------------------------
# Find out where is the header 'all model',
                    i_am = -1
                    for i in range(0, len(amh)):
                        if(amh[i][5] == 'header'):
                            if(amh[i][4].lower().find('all model') > -1) or (amh[i][4].lower().find('all models') > -1):
                                #print(amh[i][4], i)
                                i_am = i
#---------------------------------------------
                    ah = all_header(amh, i_am)
#----------------------------------------------
                    df_rev(ah)

                    mall = merge_all(ah)
#for i in range(0, len(mall)):
#    print(mall[i])
                    fout = prep_final_output(mall)
                    
                    b = pd.DataFrame(fout, columns=['Keyterm', 'Info'])
                    b.to_csv(rf'/Users/bbteela/Desktop/Work/DAEN 690/csv/{file}.csv', index=False)
                    a.append(b)
        
                #dfinal.loc[len(dfinal)] = fout
                    pdf.close()
                #for i in range(0, len(fout)):
                #    print(fout[i])
                    
                #dfinal = pd.DataFrame(fout, columns=['L', 'R'])
                #dfinal

dfinal = pd.concat(a)
no_fuel = pd.DataFrame(nof)
pdf_scan =  pd.DataFrame(scan)


/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/4A24.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/4A30.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/3A11.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/H80EU.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/A45CE.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/A23SO.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/A00004LA.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/H9WE.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/A00006WI.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/A23EU.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/A66EU.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/7A5.pdf
/Users/bbteela/Desktop/Work/DAEN 690/Aviators/Data/dataset_tcds/A00076CE.pdf
/Users/bbteela/Desktop/Work/DAEN 69

In [32]:
no_fuel

""


In [98]:
fy1

[]